### 모델 선택

### tpot

TPOTClassifier()

In [48]:
from tpot import TPOTClassifier

In [49]:
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42,scoring='f1')

In [50]:
tpot.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.45736597739025553
Generation 2 - Current best internal CV score: 0.4628555289797152
Generation 3 - Current best internal CV score: 0.4628555289797152
Generation 4 - Current best internal CV score: 0.4628555289797152
Generation 5 - Current best internal CV score: 0.46431951430432356

Best pipeline: ExtraTreesClassifier(ExtraTreesClassifier(DecisionTreeClassifier(input_matrix, criterion=gini, max_depth=3, min_samples_leaf=4, min_samples_split=13), bootstrap=True, criterion=gini, max_features=0.55, min_samples_leaf=3, min_samples_split=18, n_estimators=100), bootstrap=True, criterion=entropy, max_features=0.55, min_samples_leaf=3, min_samples_split=19, n_estimators=100)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=50,
               random_state=42, scoring='f1', subsample=1.0, template=None,
               use_dask=False, verbosity=2, warm_start=False)

In [51]:
print(tpot.score(X_train, y_train))

0.9789521998557471


In [52]:
pred_1 = tpot.predict(X_val)

In [53]:
f1_score(y_val, pred_1)

0.4613410330445942

In [54]:
pred_1

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [58]:
vars(tpot)

{'population_size': 50,
 'offspring_size': None,
 'generations': 5,
 'mutation_rate': 0.9,
 'crossover_rate': 0.1,
 'scoring': 'f1',
 'cv': 5,
 'subsample': 1.0,
 'n_jobs': 1,
 'max_time_mins': None,
 'max_eval_time_mins': 5,
 'periodic_checkpoint_folder': None,
 'early_stop': None,
 'config_dict': None,
 'template': None,
 'warm_start': False,
 'memory': None,
 'use_dask': False,
 'verbosity': 2,
 'disable_update_check': False,
 'random_state': 42,
 '_pop': [],
 '_pareto_front': <deap.tools.support.ParetoFront at 0x1c52d4f6fc8>,
 '_last_optimized_pareto_front': [0.46431951430432356,
  0.4568641307968167,
  0.4451239845743499],
 '_last_optimized_pareto_front_n_gens': 1,
 '_optimized_pipeline': [<deap.gp.Primitive at 0x1c532feec28>,
 '_optimized_pipeline_score': 0.46431951430432356,
 '_exported_pipeline_text': [],
 'fitted_pipeline_': Pipeline(memory=None,
          steps=[('stackingestimator-1',
                  StackingEstimator(estimator=DecisionTreeClassifier(class_weight=None,
   

In [94]:
tpot.export('tp.py')

In [64]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator
from tpot.export_utils import set_param_recursive

In [73]:
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeClassifier(criterion="gini", max_depth=3, min_samples_leaf=4, min_samples_split=13)),
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=True, criterion="gini", max_features=0.55, min_samples_leaf=3, min_samples_split=18, n_estimators=100)),
    ExtraTreesClassifier(bootstrap=True, criterion="entropy", max_features=0.55, min_samples_leaf=3, min_samples_split=19, n_estimators=100)
)

In [74]:
set_param_recursive(exported_pipeline.steps, 'random_state', 42)

In [75]:
exported_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('stackingestimator-1',
                 StackingEstimator(estimator=DecisionTreeClassifier(class_weight=None,
                                                                    criterion='gini',
                                                                    max_depth=3,
                                                                    max_features=None,
                                                                    max_leaf_nodes=None,
                                                                    min_impurity_decrease=0.0,
                                                                    min_impurity_split=None,
                                                                    min_samples_leaf=4,
                                                                    min_samples_split=13,
                                                                    min_weight_fraction_leaf=0.0,
                                            

In [76]:
predd = exported_pipeline.predict(X_val)

In [90]:
f1_score(y_val,predd)

0.4613410330445942

In [78]:
results = exported_pipeline.predict(test)

In [88]:
sub['fr_yn'] = results
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.head()
sub.to_csv('화재예측과제_Submission.csv', index=False)